In [1]:
import sys
!{sys.executable} -m pip install pandas

## Definim reteaua

In [2]:
import numpy as np
import torch
import torch.nn as nn
class CAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 100, kernel_size = 5)
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, return_indices=True)
        self.conv2 = nn.Conv2d(in_channels = 100, out_channels = 200, kernel_size = 5)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2, return_indices=True)
        self.maxunpool1 = nn.MaxUnpool2d(kernel_size = 2)
        self.deconnv1 = nn.ConvTranspose2d(in_channels = 200, out_channels = 100, kernel_size = 5)
        self.maxunpool2 = nn.MaxUnpool2d(kernel_size = 2)
        self.deconv2 = nn.ConvTranspose2d(in_channels = 100, out_channels = 3, kernel_size = 5)
        self.activ = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x, indices1 = self.maxpool1(x)
        x = self.activ(x)
        x = self.conv2(x)
        x, indices2 = self.maxpool2(x)
        x = self.activ(x)
        x = self.maxunpool1(x,indices2)
        x = self.deconnv1(x)
        x = self.activ(x)
        x = self.maxunpool2(x,indices1)
        x = self.deconv2(x)
        x = self.activ(x)
        return x

In [3]:
import torchvision
class ArtNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = CAN()
        self.resnet = torchvision.models.resnet18(pretrained= False)#True)
        self.relu = torch.nn.ReLU()
        self.norm1 = torch.nn.BatchNorm1d(self.resnet.fc.out_features)
        self.avg1 = torch.nn.AvgPool1d(kernel_size=4,stride=4)
        self.additional_layer1 = torch.nn.Linear(250,136)

    def forward(self,x):
        x = self.encoder(x)
        x = self.resnet(x)
        x = self.relu(x)
        x = self.norm1(x)
        x = self.avg1(x)
        x = self.additional_layer1(x)
        x = self.relu(x)
        return x

## Initializam dataloaderul

In [4]:
import pandas as pd

df = pd.read_csv(r"D:\Descarcari\Dataset\all_data_info.csv")

In [5]:
df.head(10)

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg
5,Tosa Mitsuoki,NaN,mythological painting,25528.0,3000.0,10496349.0,wikiart,Yamato-e,Night March of a Hundred Demons (left half),train_only,True,29855.jpg
6,Barnett Newman,NaN,abstract,7345.0,8640.0,6383948.0,wikiart,Color Field Painting,"Who’s Afraid of Red, Yellow and Blue II",train_only,True,62252.jpg
7,Hiroshige,1838,bird-and-flower painting,6483.0,9276.0,12762418.0,wikiart,Ukiyo-e,Small Bird on a Branch of Kaidozakura,train_and_test,False,49823.jpg
8,Barnett Newman,1963.0,abstract,6049.0,8206.0,1135960.0,wikiart,Color Field Painting,Black Fire I,train_only,True,63861.jpg
9,Franz Richard Unterberger,NaN,cityscape,6238.0,7352.0,7896792.0,wikiart,Romanticism,Procession in Naples,train_and_test,False,84512.jpg


In [6]:
list_of_styles = list(pd.unique(df['style']))

In [7]:
df = df.drop(columns = ['artist','date','genre','pixelsx','pixelsy','size_bytes','source','title'])
#df.drop('date')
#df.drop('genre')
#df.drop('pixelsx')
#df.drop('pixelsy')
#df.drop('size_byte')
#df.drop('source')
#df.drop('title')


In [8]:
print(df.dtypes)

style           object
artist_group    object
in_train          bool
new_filename    object
dtype: object


In [9]:

df_train = df.loc[df['in_train']]
df_test = df.loc[df['artist_group'] != 'train_only']

In [10]:
from torchvision.io import read_image
import os
class PaintingDataset(torch.utils.data.Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.abspath(os.path.join(self.img_dir,self.img_labels.iloc[idx,3]).replace('jpg','png'))
        #print(img_path)
        try:
            image = read_image(img_path)
        except:
            return None
        label = list_of_styles.index(self.img_labels.iloc[idx, 0])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [11]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize,Grayscale
from torchvision.transforms.functional import to_tensor,  normalize

my_transform = Normalize([0.5], [0.5])
grayscale_prevent = Grayscale(num_output_channels = 3)
def my_collate(examples):
    images = []
    labels = []
    examples = list(filter(lambda x: x is not None, examples))
    for example in examples:
        image = example[0]
        if image.shape != (3, 256, 256):
            continue
        image = image.float()
        image = image.unsqueeze(0)
        image = my_transform(image)
        images.append(image)
        label = np.array([example[1]])
        label = torch.Tensor(label)
        label = label.unsqueeze(0)
        labels.append(label)
    images = torch.cat(images,  dim=0)
    labels = torch.cat(labels, dim=0)
    return images, labels

In [12]:
training_data = PaintingDataset(df_train,'D:\Descarcari\Dataset',transform = torchvision.transforms.Resize(size = (256,256)))
testing_data = PaintingDataset(df_test,'D:\Descarcari\Dataset',transform = torchvision.transforms.Resize(size = (256,256)))



In [13]:
train_dataloader = DataLoader(training_data, batch_size=5, shuffle=False,collate_fn=my_collate)
test_dataloader = DataLoader(testing_data, batch_size=5, shuffle=False,collate_fn=my_collate)

### Implementarea functiei de loss

In [14]:
class my_loss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cos = nn.CosineSimilarity(dim=0)

    def forward(self,out,labels):
        out_list = list(torch.unbind(out,dim = 0))
        labels_list = list(torch.unbind(labels,dim = 0))
        total = 0
        counts = 0
        for i in range(len(out_list)+1):
            for j in range(i):
                if i != j:
                    counts = counts + 1
                    labels = 1 if labels_list[i-1]==labels_list[j] else -1
                total = total +(labels-self.cos(out_list[i-1],out_list[j]))**2
        return total/counts
    

## Definim antrenarea 

In [15]:
import torch.optim as optim

epochs = 75

net = ArtNet()
net.cuda()

optimiser = optim.Adam(params= net.parameters(),lr = 1e-4)
optimiser.zero_grad()

loss = my_loss()


In [16]:
from itertools import combinations

#precision may vary depending on what someone else would consider enough similarity/disimilarity to draw a conclusion
def test_acc(net: nn.Module, test_loader: DataLoader):
    net.eval()
    cos = nn.CosineSimilarity(dim=1,eps = 1e-6)
    total = 0
    correct = 0
    batchnr = 1
    for test_images, test_labels in test_loader:
        test_images = test_images.cuda()
        test_labels = test_labels.cuda()
        images_list = list(torch.unbind(test_images,dim = 0))
        labels_list = list(torch.unbind(test_labels,dim = 0))
        for i in range(len(images_list)+1):
            for j in range(i):
                if i != j:
                    total = total + 1
                    should_be_similar = True if labels_list[i-1] == labels_list[j] else False
                    proximity = cos(net(images_list[i-1].unsqueeze(0)),net(images_list[j].unsqueeze(0)))
                    if (proximity < -0.9 and not should_be_similar) or (proximity > 0.9 and should_be_similar):
                        correct = correct + 1
        if batchnr  == 100: 
            break
        batchnr = batchnr + 1
    return correct / total * 100


def train(epochs: int, train_loader: DataLoader, test_loader: DataLoader, net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
    for e in range(epochs):
        net.train()
        batchnr = 1
        for images, labels in train_loader:
            images = images.cuda()
            labels = labels.cuda()
            out = net(images)
            labels = torch.reshape(labels,(-1,)).long()
            loss = loss_fn(out,labels)
            batchnr = batchnr +1
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if batchnr  == 500:
                #print('batchul '+str(batchnr)+' epoca '+str(e)+' are loss '+str(loss.item()))
                print("Loss-ul la finalul epocii {} are valoarea {}".format(e + 1, loss.item()))
                break
        acc = test_acc(net, test_loader)
        print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
cuda = torch.device('cuda')
torch.cuda.empty_cache()
train(epochs, train_dataloader, test_dataloader, net, loss, optimiser)

D:\Utilitati\Anaconda\lib\site-packages\torch\nn\functional.py:749: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Loss-ul la finalul epocii 1 are valoarea 0.7600237131118774
Acuratetea la finalul epocii 1 este 34.48%
Loss-ul la finalul epocii 2 are valoarea 0.7330319881439209
Acuratetea la finalul epocii 2 este 34.75%
Loss-ul la finalul epocii 3 are valoarea 0.764802098274231
Acuratetea la finalul epocii 3 este 34.28%
Loss-ul la finalul epocii 4 are valoarea 0.6846963763237
Acuratetea la finalul epocii 4 este 34.34%
Loss-ul la finalul epocii 5 are valoarea 0.7704229354858398
Acuratetea la finalul epocii 5 este 33.94%
Loss-ul la finalul epocii 6 are valoarea 0.7617591619491577
Acuratetea la finalul epocii 6 este 39.80%
Loss-ul la finalul epocii 7 are valoarea 0.6828377842903137
Acuratetea la finalul epocii 7 este 35.02%
Loss-ul la finalul epocii 8 are valoarea 0.7334938645362854
Acuratetea la finalul epocii 8 este 33.80%
Loss-ul la finalul epocii 9 are valoarea 0.7003701329231262
Acuratetea la finalul epocii 9 este 33.74%
Loss-ul la finalul epocii 10 are valoarea 0.7171266674995422
Acuratetea la fi